In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import backend as K
import numpy as np
import os
import time
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session


config = tf.ConfigProto()
config.gpu_options.allow_growth = True  #dynamically grow the memory used on the GPU
set_session(tf.Session(config=config))

# dimensions of our images.
img_width, img_height = 512, 512

train_data_dir = 'E:\\UAV_drone\\data\\training\\vertical_view\\train'
validation_data_dir = 'E:\\UAV_drone\\data\\training\\vertical_view\\val'
nb_train_samples = 50041
nb_validation_samples = 12117
epochs = 100
batch_size = 16
img_channels = 3
nb_classes = 3

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1. / 255)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(512, 512),
    batch_size=batch_size,
    classes=['down', 'medium', 'up'])

label_map = (train_generator.class_indices)
print(label_map)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(512, 512),
    batch_size=batch_size,
    classes=['down', 'medium', 'up'])


# checkpoint
current_directory = os.getcwd()

checkpoint_dir = os.path.join(current_directory, 'checkpoints')
result_dir = os.path.join(current_directory, 'results')

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

Using TensorFlow backend.


Found 50041 images belonging to 3 classes.
{'down': 0, 'medium': 1, 'up': 2}
Found 12117 images belonging to 3 classes.


### Load base model

In [2]:
from keras.models import load_model    

model_file = 'model_mobilenetv2_view-offset-altitude_08.hdf5'
model_full_path = os.path.join(result_dir, model_file)
model = load_model(model_full_path)

model.summary()

# weights = model.get_weights()
# print(len(weights))
# print(len(model.layers))
# print(len(weights[:-4]))
# for i in range(0, len(weights)):
#     print(weights[i].shape)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 513, 513, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 256, 256, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 256, 256, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

### Transfering to new model

In [3]:
from keras.models import Model
new_model = Model(model.inputs, model.layers[-3].output)

for i in range(0, (len(new_model.layers)-2)):
    new_model.layers[i].trainable = False
    
new_model.summary()
new_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

weights = model.get_weights()
new_model.set_weights(weights[:-4])

# delete(model)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 513, 513, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 256, 256, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 256, 256, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [4]:
#check if weights are loaded into the new_model
new_model.get_weights()

[array([[[[-0.05274788,  0.7343213 , -0.40835723, -0.19860865,
           -0.03321066, -0.44020975, -0.03313436,  0.30878252,
           -0.04252747,  0.2153353 , -0.04581205,  0.06359087,
            0.13275188,  0.09275927,  0.33060455,  0.19295233,
            0.17830147,  0.0711166 ,  0.09956403,  0.33816326,
            0.5667758 ,  0.05869669, -0.206742  ,  0.14737354,
           -0.4371786 ,  0.00915318, -0.03473896, -0.05904517,
            0.05645791, -0.07455492, -0.16718905,  0.01822209],
          [-0.05307516,  0.24810301, -0.38990787,  0.32114422,
           -0.19177514, -0.14824545,  0.06514753,  0.20443156,
            0.00606214, -0.03432251, -0.15018025, -0.00203701,
            0.03400833, -0.20662397,  0.19636185,  0.21512467,
            0.17507073,  0.05227491, -0.07929025, -0.02637791,
            0.18752573, -0.0496781 , -0.3913458 ,  0.02193481,
           -0.41818786, -0.07624657, -0.08902517, -0.3397264 ,
           -0.13876644,  0.32346094, -0.10896841,  0.1

### Training the transfered-model

In [5]:
file="model_mobilenetv2_cam-pitch_transfer-learning_{epoch:02d}.hdf5"
checkpoint =  ModelCheckpoint(os.path.join(checkpoint_dir, file), verbose=1)

NAME = "mobilenetv2_cam-pitch_transfer-learning_{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME), write_graph = True, write_images=True)

new_model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpoint, tensorboard])

Epoch 1/100
3127/3127 [==============================] - 1656s 530ms/step - loss: 0.7448 - acc: 0.8312 - val_loss: 0.2904 - val_acc: 0.9020

Epoch 00001: saving model to C:\Users\hoang\WorkingSpace\TrainingModels\keras\checkpoints\model_mobilenetv2_cam-pitch_transfer-learning_01.hdf5
Epoch 2/100
3127/3127 [==============================] - 1682s 538ms/step - loss: 0.1332 - acc: 0.9572 - val_loss: 0.2378 - val_acc: 0.9212

Epoch 00002: saving model to C:\Users\hoang\WorkingSpace\TrainingModels\keras\checkpoints\model_mobilenetv2_cam-pitch_transfer-learning_02.hdf5
Epoch 3/100
3127/3127 [==============================] - 1726s 552ms/step - loss: 0.1056 - acc: 0.9665 - val_loss: 0.2130 - val_acc: 0.9335

Epoch 00003: saving model to C:\Users\hoang\WorkingSpace\TrainingModels\keras\checkpoints\model_mobilenetv2_cam-pitch_transfer-learning_03.hdf5
Epoch 4/100
3127/3127 [==============================] - 1710s 547ms/step - loss: 0.0903 - acc: 0.9703 - val_loss: 0.1913 - val_acc: 0.9381

Epoc

KeyboardInterrupt: 